In [269]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

In [270]:
# Path to the original data.pkl file
data_path = "D:\Knowledge\Hiwi\python-chebai\data\chebi_v231\ChEBI50\processed\data.pkl"
# data_path1="data\chebi_v231\ChEBI50\processed\data.pkl"

In [271]:
df = pd.read_pickle(
    open(data_path, "rb"
    )
)
df[:5]

,id,name,SMILES,1722,2440,2468,2571,2580,2634,3098,...,176910,177333,183508,183509,189832,189840,192499,194321,197504,229684
0,33429,monoatomic monoanion,[*-],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,30151,aluminide(1-),[Al-],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,16042,halide anion,[*-],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,17051,fluoride,[F-],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,28741,sodium fluoride,[F-].[Na+],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [272]:
result = df[df["id"] == 5385]
result

,id,name,SMILES,1722,2440,2468,2571,2580,2634,3098,...,176910,177333,183508,183509,189832,189840,192499,194321,197504,229684
31356,5385,gliotoxin,[H][C@@]12[C@@H](O)C=CC=C1C[C@@]13SS[C@@](CO)(...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [273]:
# Create a new empty DataFrame for storing new variations
new_df = pd.DataFrame(columns=df.columns)


In [274]:
# Function to generate SMILES variations using different configurations
def generate_smiles_variations(smiles, num_variations=5):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return []  # Return an empty list if conversion fails

    variations = set()

    # Loop through all combinations of doRandom and rootedAtAtom values
    for do_random in [True, False]:
        for rooted_at_atom in [5, 4, 3, 2, 1, 0, -1, -2, -3, -4, -5]:
            try:
                # Generate SMILES with the given configuration
                variant = Chem.MolToSmiles(mol, doRandom=do_random, rootedAtAtom=rooted_at_atom)
                if variant != smiles:  # Avoid duplicates with the original SMILES
                    variations.add(variant)

                # Check the number of variations after adding
                if len(variations) >= num_variations:
                    return list(variations)  # Return immediately when enough variations are found

            except Exception as e:
                # Skip invalid configurations
                continue

    return list(variations)

In [275]:
# smile="C(CC[N+]1(C)CCCC1)(O)(C2CCCCC2)C3=CC=CC=C3.[Cl-]"
# sample_variation=generate_smiles_variations(smile)

In [276]:
# sample_variation

In [277]:
# Set to keep track of already seen SMILES to avoid duplicates
seen_smiles = set(df['SMILES'])

In [278]:
test_df=df[-5::]

In [279]:
test_df

,id,name,SMILES,1722,2440,2468,2571,2580,2634,3098,...,176910,177333,183508,183509,189832,189840,192499,194321,197504,229684
185011,229518,2-Amino-3-methylsuccinic acid,OC(=O)C(C(N)C(O)=O)C,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
185012,83380,dinocap-4,C\C=C\C(=O)Oc1c(cc([*])cc1[N+]([O-])=O)[N+]([O...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
185013,140503,kaolin,[Al](O[Si](O[Si](O[Al]=O)=O)=O)=O.O.O,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
185014,81948,tralkoxydim,CCO\N=C(CC)\C1=C(O)CC(CC1=O)c1c(C)cc(C)cc1C,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
185015,140499,kaolinite,[OH-].[OH-].[OH-].[OH-].O=[Si]([O-])O[Si](=O)[...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [280]:
test_df.nunique()

id        5
name      5
SMILES    5
1722      1
2440      1
         ..
189840    1
192499    1
194321    1
197504    1
229684    1
Length: 1514, dtype: int64

In [281]:
# Process each row in the original DataFrame
for _, row in test_df.iterrows():
    original_smiles = row['SMILES']
    
    # Generate new SMILES variations
    variations = generate_smiles_variations(original_smiles)
    
    # Filter out variations that are already seen
    variations = [var for var in variations if var not in seen_smiles]
    
    for var in variations:
        # Create a new row with the new SMILES and the rest of the features and labels unchanged
        new_row = row.copy()
        new_row['SMILES'] = var
        new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
        
        # Add the new SMILES to the seen set to avoid duplicates
        seen_smiles.add(var)

[20:01:38] 

****
Range Error
idx
Violation occurred on line 209 in file C:\rdkit\build\temp.win-amd64-cpython-311\Release\rdkit\Code\GraphMol\ROMol.cpp
Failed Expression: 5 < 1
****

[20:01:38] 

****
Range Error
idx
Violation occurred on line 209 in file C:\rdkit\build\temp.win-amd64-cpython-311\Release\rdkit\Code\GraphMol\ROMol.cpp
Failed Expression: 4 < 1
****

[20:01:38] 

****
Range Error
idx
Violation occurred on line 209 in file C:\rdkit\build\temp.win-amd64-cpython-311\Release\rdkit\Code\GraphMol\ROMol.cpp
Failed Expression: 3 < 1
****

[20:01:38] 

****
Range Error
idx
Violation occurred on line 209 in file C:\rdkit\build\temp.win-amd64-cpython-311\Release\rdkit\Code\GraphMol\ROMol.cpp
Failed Expression: 2 < 1
****

[20:01:38] 

****
Range Error
idx
Violation occurred on line 209 in file C:\rdkit\build\temp.win-amd64-cpython-311\Release\rdkit\Code\GraphMol\ROMol.cpp
Failed Expression: 1 < 1
****

[20:01:38] 

****
Range Error
idx
Violation occurred on line 209 in file C:\rdki

In [282]:
new_df.nunique()

id         5
name       5
SMILES    24
1722       1
2440       1
          ..
189840     1
192499     1
194321     1
197504     1
229684     1
Length: 1514, dtype: int64

In [283]:
# Append the new DataFrame (new_df) to the original DataFrame (df)
df_combined = pd.concat([test_df, new_df], ignore_index=True)

In [284]:
df_combined.shape

(29, 1514)

In [285]:
new_data_path="test_data.pkl"

In [286]:
pd.to_pickle(df_combined, open(new_data_path, "wb"))

In [287]:
test_data_df= pd.read_pickle(
    open("test_data.pkl", "rb"
    )
)

In [288]:
test_data_df

,id,name,SMILES,1722,2440,2468,2571,2580,2634,3098,...,176910,177333,183508,183509,189832,189840,192499,194321,197504,229684
0,229518,2-Amino-3-methylsuccinic acid,OC(=O)C(C(N)C(O)=O)C,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,83380,dinocap-4,C\C=C\C(=O)Oc1c(cc([*])cc1[N+]([O-])=O)[N+]([O...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,140503,kaolin,[Al](O[Si](O[Si](O[Al]=O)=O)=O)=O.O.O,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,81948,tralkoxydim,CCO\N=C(CC)\C1=C(O)CC(CC1=O)c1c(C)cc(C)cc1C,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,140499,kaolinite,[OH-].[OH-].[OH-].[OH-].O=[Si]([O-])O[Si](=O)[...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,229518,2-Amino-3-methylsuccinic acid,O=C(C(C(C(O)=O)N)C)O,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,229518,2-Amino-3-methylsuccinic acid,C(C(C(N)C(=O)O)C)(=O)O,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,229518,2-Amino-3-methylsuccinic acid,NC(C(=O)O)C(C(O)=O)C,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,229518,2-Amino-3-methylsuccinic acid,C(C(C(O)=O)C)(C(O)=O)N,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,229518,2-Amino-3-methylsuccinic acid,C(C(C(O)=O)N)(C(O)=O)C,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [289]:
test_data_df.shape

(29, 1514)

In [290]:
test_data_df.nunique()

id         5
name       5
SMILES    29
1722       1
2440       1
          ..
189840     1
192499     1
194321     1
197504     1
229684     1
Length: 1514, dtype: int64

Each time we run it is generating different number of new variations